In [41]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import numpy as np

In [42]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="food_model_combined_v2.tflite")
interpreter.allocate_tensors()

In [43]:
# Load the dataset info
food101_info = tfds.builder('food101').info

In [44]:
# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [45]:
# Function to preprocess the image
def preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    return img

In [46]:
# Load and preprocess a test image
image_path = 'carrot_cake.png'
image = preprocess_image(image_path)
image = tf.expand_dims(image, 0)  

In [47]:
# Set the tensor to point to the input data to be inferred
interpreter.set_tensor(input_details[0]['index'], image)

In [48]:
# Run the inference
interpreter.invoke()

In [49]:
# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])


In [50]:
# Get the predicted class
predicted_class = np.argmax(output_data)

In [51]:
# Get class names
food101_class_names = food101_info.features['label'].names
print("Food101 classes:", len(food101_class_names))
print("First few Food101 classes:", food101_class_names[:5])

Food101 classes: 101
First few Food101 classes: ['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare']


In [52]:
fruits360_dir = 'fruits-360_dataset_100x100/fruits-360/Training'

# Get class names from directory names
fruits360_class_names = sorted(os.listdir(fruits360_dir))

print("Fruits-360 classes:", len(fruits360_class_names))
print("First few Fruits-360 classes:", fruits360_class_names[:5])

Fruits-360 classes: 141
First few Fruits-360 classes: ['Apple 6', 'Apple Braeburn 1', 'Apple Crimson Snow 1', 'Apple Golden 1', 'Apple Golden 2']


In [53]:
# Combine the class names
all_class_names = food101_class_names + fruits360_class_names

print("Total classes:", len(all_class_names))
print("First few combined classes:", all_class_names[:5])
print("Last few combined classes:", all_class_names[-5:])

Total classes: 242
First few combined classes: ['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare']
Last few combined classes: ['Tomato not Ripened 1', 'Walnut 1', 'Watermelon 1', 'Zucchini 1', 'Zucchini dark 1']


In [54]:
loaded_class_names = np.load('combined_class_names.npy', allow_pickle=True)

In [55]:
def predict_class(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    
    predictions = model.predict(img_array)
    predicted_class = loaded_class_names[np.argmax(predictions[0])]
    confidence = np.max(predictions[0])
    
    print(f"Predicted class: {predicted_class}")
    print(f"Confidence: {confidence:.2f}")

In [56]:
# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

In [57]:
# Get the predicted class
predicted_class_index = np.argmax(output_data)

In [58]:
# Print the predicted class and its probability
print(f"Predicted class index: {predicted_class_index}")
print(f"Confidence: {output_data[0][predicted_class_index]:.4f}")

Predicted class index: 14
Confidence: 0.8897


In [59]:
# Get the predicted class name
predicted_class_name = all_class_names[predicted_class_index]
print(f"Predicted class name: {predicted_class_name}")

Predicted class name: carrot_cake


In [60]:
# Print top 5 predictions
top_5_indices = np.argsort(output_data[0])[-5:][::-1]
print("\nTop 5 predictions:")
for i, idx in enumerate(top_5_indices, 1):
    print(f"{i}. {all_class_names[idx]}: {output_data[0][idx]:.4f}")


Top 5 predictions:
1. carrot_cake: 0.8897
2. churros: 0.0288
3. macarons: 0.0252
4. donuts: 0.0165
5. fried_calamari: 0.0099
